In [1]:
import pandas as pd
import numpy as np

In [2]:
#urls = 'https://github.com/dragon-library/work_space/raw/main/HS_Code/HS/hs_code.xlsx'
urls = 'data/hs_code.xlsx'
types = 'section'
#types = "chapter"

def get_master(sheets,types = 'section'):
    data = pd.read_excel(urls,sheet_name= sheets)
    data[types] = data[types].map('{:02}'.format)
    data = data[[types,'description']]
    data['description'] = data['description'].str.lower()
 #   data = data.rename(columns={'heading' : 'target', 'product_desc' : 'question_text'})
     

    return data

def manage_data(df):
    df.columns = ['target', 'data']   
    
    
    return df

import pickle
# save the model to disk
def save_model(model,filename = "data/finalized_model.sav"):
    pickle.dump(model, open(filename, 'wb'))
    
    print('Save Model Success')
 

In [3]:
#types = "section"
#types = "chapter"


sheets = '8_digit'
eights = get_master(sheets,types)
sheets = '6_digit'
sixs = get_master(sheets,types)
sheets = '4_digit'
fours = get_master(sheets,types)
sheets = '2_digit'
twos = get_master(sheets,types)

sheets = 'test_01'
tests = get_master(sheets,types)

data = pd.concat([eights,sixs,fours,twos], ignore_index=True)
twenty_train = manage_data(data)


twenty_test = manage_data(tests)

twenty_train.head()

,target,data
0,01,"horses; live, purebred breeding animals - pure..."
1,01,"horses; live, other than purebred breeding ani..."
2,01,asses; live - other
3,01,mules and hinnies; live- other
4,01,"cattle; live, purebred breeding animals - pure..."


In [4]:
twenty_test.head()

,target,data
0,01,live purebred breeding horses
1,01,live horses other than purebred breeding horses
2,01,live asses
3,01,mules and hinnies imported for immediate slaug...
4,01,mules and hinnies not imported for immediate s...


In [5]:
arr_class = twenty_train.target.unique()
arr_class 

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '00'],
      dtype=object)

In [6]:
print(twenty_train.data[5])

cattle; live, other than purebred breeding animals -  oxen


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
print(X_train_counts.shape)

(17223, 8535)


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(17223, 8535)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier 
import numpy as np

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3,  random_state=42)),])
text_clf_svm.fit(twenty_train.data, twenty_train.target)
predicted_svm = text_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.8770428681440917

In [10]:
from sklearn.model_selection import GridSearchCV #ใช้วิธี กาเดี้ยนดีเซ้น
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf-svm__alpha': (1e-2, 1e-3),}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
save_model(gs_clf_svm)
gs_clf_svm.best_score_ 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Save Model Success


0.952273270524536

In [11]:
gs_clf_svm.best_params_ # Param 

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [12]:
predicted_gs_svm = gs_clf_svm.predict(twenty_test.data)
np.mean(predicted_gs_svm == twenty_test.target) #คะแนน TEST จริงๆ

0.87635834688115

In [13]:
gs_clf_svm.best_score_ 

0.952273270524536

In [21]:
gs_clf_svm

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf-svm',
                                        SGDClassifier(alpha=0.001,
                                                      random_state=42))]),
             n_jobs=-1,
             param_grid={'clf-svm__alpha': (0.01, 0.001),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2)]})

In [14]:
from sklearn.metrics import classification_report
print (classification_report(twenty_test.target, predicted_svm))



              precision    recall  f1-score   support

          01       0.79      0.90      0.84       738
          02       0.86      0.89      0.88       594
          03       0.82      0.83      0.83        71
          04       0.94      0.69      0.80       883
          05       0.86      0.71      0.78       204
          06       0.87      0.93      0.90      1833
          07       0.84      0.86      0.85       407
          08       0.98      0.83      0.90       231
          09       0.92      0.82      0.86       343
          10       0.91      0.97      0.94       275
          11       0.92      0.99      0.95      1696
          12       0.96      0.77      0.86       206
          13       0.90      0.86      0.88       342
          14       0.90      0.65      0.76       127
          15       0.88      0.97      0.92      1050
          16       0.83      0.95      0.88      1478
          17       0.77      0.89      0.82       280
          18       0.97    